# 设备服务设计

## 设备知识管理
>智盒服务提供知识的添加、删除、查询接口(目前只实现了知识查询接口),设备根据自身的运行状态和所具有的知识来产生事件,并上报给规则引擎和平台。

### 事件格式
>设备通过知识产生的事件格式如下

```json
{
  "productIdentifier": "D6_ZK_LC_6B11_J",
  "devSn": "dev1",
  "eventIdentifier": "lightOn",
  "params": {
    "onOff": 1
  }
}
```
### 设备知识格式
>目前每个设备的知识固化在xml文件中。格式如下

```xml
<?xml version="1.0" encoding="UTF-8"?>
<knowledge>
    <Universal>
        <unusualLighting>
            <name>异常照明事件</name>
            <propertyInfo>
                <property>brightness</property>
                <operator>IN</operator>
                <value>(0 20)</value>
            </propertyInfo>
        </unusualLighting>
        <energySavingLighting>
            <name>节能照明事件</name>
            <propertyInfo>
                <property>brightness</property>
                <operator>IN</operator>
                <value>[20 50)</value>
            </propertyInfo>
        </energySavingLighting>
        <normalLighting>
            <name>正常照明事件</name>
            <propertyInfo>
                <property>brightness</property>
                <operator>IN</operator>
                <value>[50 80)</value>
            </propertyInfo>
        </normalLighting>
        <highlightLighting>
            <name>高亮照明事件</name>
            <propertyInfo>
                <property>brightness</property>
                <operator>GTE</operator>
                <value>80</value>
            </propertyInfo>
        </highlightLighting>
        <lightOut>
            <name>关灯事件</name>
            <propertyInfo>
                <property>onOff</property>
                <operator>E</operator>
                <value>0</value>
            </propertyInfo>
        </lightOut>
        <lightOn>
            <name>开灯事件</name>
            <propertyInfo>
                <property>onOff</property>
                <operator>E</operator>
                <value>1</value>
            </propertyInfo>
        </lightOn>
    </Universal>
</knowledge>
```


## 定时策略管理
>每个设备提供添加、删除定时策略的服务

### 定时策略报文
```json
{
  "rule": {
    "uuid": "1",
    "enable": 1,
    "date": {
      "startDate": "2022-09-26",
      "endDate": "2022-10-28"
    },
    "times": [
      {
        "startTime": "23:26:00",
        "endTime": "20:23:23",
        "params": {
          "onOff": 1,
          "brightness": 30
        }
      },
      {
        "startTime": "22:00:00",
        "endTime": "23:23:23",
        "params": {
          "onOff": 1,
          "brightness": 30
        }
      }
    ]
  }
}
```

### 添加定时策略

```json
{
  "id": "addRule",
  "name": "添加定时策略",
  "output": [],
  "parameters": [
    {
      "id": "rule",
      "name": "规则",
      "type": "struct",
      "struct": [
        {
          "id": "uuid",
          "name": "规则ID",
          "type": "string"
        },
        {
          "id": "enable",
          "name": "是否使能",
          "type": "integer",
          "specs": {
            "optional": [
              {
                "desc": "使能",
                "value": 1
              },
              {
                "desc": "失能",
                "value": 0
              }
            ]
          }
        },
        {
          "id": "date",
          "name": "日期",
          "type": "struct",
          "struct": [
            {
              "id": "startDate",
              "name": "开始日期",
              "format": "date",
              "type": "string"
            },
            {
              "id": "endDate",
              "name": "结束日期",
              "format": "date",
              "type": "string"
            }
          ]
        },
        {
          "id": "times",
          "name": "时间列表",
          "required": true,
          "type": "array",
          "columnComplex": [
            {
              "id": "startTime",
              "name": "开始时间",
              "format": "time",
              "type": "string"
            },
            {
              "id": "endTime",
              "name": "结束时间",
              "format": "time",
              "type": "string"
            },
            {
              "id": "params",
              "name": "动作参数",
              "type": "struct",
              "struct": [
                {
                  "id": "onOff",
                  "name": "开关",
                  "type": "integer",
                  "specs": {
                    "optional": [
                      {
                        "desc": "使能",
                        "value": 1
                      },
                      {
                        "desc": "失能",
                        "value": 0
                      }
                    ]
                  }
                },
                {
                  "id": "brightness",
                  "name": "亮度",
                  "type": "integer",
                  "required": false,
                  "specs": {
                    "max": 100,
                    "min": 0,
                    "step": 1
                  }
                }
              ]
            }
          ]
        }
      ]
    }
  ]
}
```

### 删除定时策略

```json
 {
  "id": "removeRules",
  "name": "删除定时规则",
  "output": [],
  "parameters": [
    {
      "id": "rules",
      "name": "规则id列表",
      "type": "array",
      "columnSimple": {
        "type": "string"
      }
    }
  ]
}
```

### 查询定时策略

```json
    {
  "id": "getRules",
  "name": "获取定时规则",
  "output": [
    {
      "id": "rules",
      "name": "规则id列表",
      "type": "array",
      "columnComplex": [
        {
          "id": "uuid",
          "name": "规则ID",
          "type": "string"
        },
        {
          "id": "enable",
          "name": "是否使能",
          "type": "integer",
          "specs": {
            "optional": [
              {
                "desc": "开",
                "value": 1
              },
              {
                "desc": "关",
                "value": 0
              }
            ]
          }
        },
        {
          "id": "date",
          "name": "日期",
          "type": "struct",
          "struct": [
            {
              "id": "startDate",
              "name": "开始日期",
              "format": "date",
              "type": "string"
            },
            {
              "id": "endDate",
              "name": "结束日期",
              "format": "date",
              "type": "string"
            }
          ]
        },
        {
          "id": "times",
          "name": "时间列表",
          "required": true,
          "type": "array",
          "columnComplex": [
            {
              "id": "startTime",
              "name": "开始时间",
              "format": "time",
              "type": "string"
            },
            {
              "id": "endTime",
              "name": "结束时间",
              "format": "time",
              "type": "string"
            },
            {
              "id": "params",
              "name": "动作参数",
              "type": "struct",
              "struct": [
                {
                  "id": "onOff",
                  "name": "开关",
                  "type": "integer",
                  "specs": {
                    "optional": [
                      {
                        "desc": "开",
                        "value": 1
                      },
                      {
                        "desc": "关",
                        "value": 0
                      }
                    ]
                  }
                },
                {
                  "id": "brightness",
                  "name": "亮度",
                  "type": "integer",
                  "required": false,
                  "specs": {
                    "max": 100,
                    "min": 0,
                    "step": 1
                  }
                }
              ]
            }
          ]
        }
      ]
    }
  ],
  "parameters": []
}
```
## 定时策略运行逻辑
>自动模式下设备服务每分钟检查一次自身的定时策略，如果当前时间点在某个定时策略的时间范围内，则向规则引擎发送定时开始事件,反之则向规则引擎发送默认定时事件.

### 定时事件的格式

```json
{
  "productIdentifier": "D6_ZK_LC_6B11_J",
  "devSn": "dev1",
  "eventIdentifier": "TIMER_START",
  "params": {
    "onOff": 80,
    "brightness": 50
  }
}
```

```json
{
  "productIdentifier": "D6_ZK_LC_6B11_J",
  "devSn": "dev1",
  "eventIdentifier": "TIMER_DEFAULT",
  "params": {
    "onOff": 1,
    "brightness": 100
  }
}
```

## 自动模式下的手动控制
>设备在自动模式下,用户向设备服务发送业务指令，则设备服务会向规则引擎发送手动控制事件.

### 手动控制事件格式

```json
{
  "productIdentifier": "D6_ZK_LC_6B11_J",
  "devSn": "dev1",
  "eventIdentifier": "MANUAL_SERVICE",
  "serviceIdentifier":"setBrightness",
  "params": {
    "brightness": 50
  }
}
```
